# 9. GAN
This section will be an exercise. Surprisingly, you can build GAN fairly easily just by using the concepts we learned so far.

## Preparation
Read section 1, 2, 3 of the original [GAN paper](https://arxiv.org/pdf/1406.2661.pdf). Then, follow the next 7 steps to implement GAN.

To summarie, we have the following problem setup:
- $x$: data with distribution $p_{data}$
- $p_g$: distribution trained by the generator
- $z$: prior input noise variables
- $p_z$: prior of $z$
- $G(z;\theta_G)$: generator neural network with parameter $\theta_G$
- $D(z;\theta_D)$: discriminator neural network with parameter $\theta_D$

The goal for $D,G$ are the following:
- $D$: $max_D V(D) = E_{x\sim p_{data}}(x)[logD(x)] + E_{z\sim p_z(z)}[log(1-D(G(z))]$
- $G$: $min_G V(G) = E_{z\sim p_z(z)}[log(1-D(G(z))]$

In [1]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


We pre-selected the hyper parameters for you this time. You usually need to tune this yourself.

In [2]:
# Training Params
num_steps = 500000
batch_size = 128
learning_rate = 0.0002

The hidden dimensions of the generator and the discriminator are also prespecified.

In [3]:
# Network Params
image_dim = 784 # 28*28 pixels
gen_hidden_dim = 256
disc_hidden_dim = 256
noise_dim = 100 # Noise data points

In [4]:
tf.reset_default_graph() # Clearing all tensors before this

### 1. Implement generator and discriminator.

Let both G and D be a 1-hidden layer fully connected neural network. Use ReLU for activation function for the hidden layer. For out layer, you should know what to use :)
Since G and D are both 1-hidden layer fully connected NN, we need 
- weights for hidden layer and out layer
- bias for hidden layer and out layer

for each. Use `tf.layers.dense`.

Becareful about the dimensions of the layers:
- $G$ takes in noise and generates an image.
- $D$ takes in an image and outputs a probability of the image being real.

In [5]:
# Generator
def generator(noises, reuse=False):
    with tf.variable_scope('generator') as scope:
        if (reuse):
            tf.get_variable_scope().reuse_variables()
        # hidden layer with name "g_hidden"
        hidden = tf.layers.dense(noises, gen_hidden_dim, tf.nn.relu, name='g_hidden')
        # out layer with name "g_out"
        out_images = tf.layers.dense(hidden, image_dim, tf.nn.sigmoid, name='g_out')
    return out_images

# Discriminator
def discriminator(images, reuse=False):
    with tf.variable_scope('discriminator') as scope:
        if (reuse):
            tf.get_variable_scope().reuse_variables()            
        # hidden layer with name "d_hidden"
        hidden = tf.layers.dense(images, disc_hidden_dim, tf.nn.relu, name='d_hidden')
        # out layer with name "d_out"
        out_prob = tf.layers.dense(hidden, 1, tf.nn.sigmoid, name='d_out')
    return out_prob

### 2. Define the inputs to generator and discriminator.
- Input to G: (batch size) $\times$ ??
- Input to D: (batch size) $\times$ ??

Think about what ?? should be.

In [6]:
gen_input = tf.placeholder(tf.float32, shape=[None, noise_dim], name='input_noise')
disc_input = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input')

### 3. Input noise to G and generate images.
This should be a one linear.

In [7]:
gen_sample = generator(gen_input)

### 3. Input real and fake images to D and get predictions.
For D, you should have two inputs: real data and fake data. The latter is the output of $G$. For the latter, set `reuse=True`. I won't go into detail about it, but basically, you are reusing the samve variables in the above `discriminator` function and so you want to make them reusable.

In [8]:
disc_real = discriminator(disc_input)
disc_fake = discriminator(gen_sample, reuse=True)

In [12]:
### 4. Define the objective.
Expectation should be approximated using the sample mean. As a reminder, they are:
- $D$: $max_D V(D) = E_{x\sim p_{data}}(x)[logD(x)] + E_{z\sim p_z(z)}[log(1-D(G(z))]$
- $G$: $min_G V(G) = E_{z\sim p_z(z)}[log(1-D(G(z))]$

In [9]:
gen_loss = -tf.reduce_mean(tf.log(disc_fake))
disc_loss = -tf.reduce_mean(tf.log(disc_real) + tf.log(1. - disc_fake))

### 5. Minimize (or maximize) the objective.
Adam optimizer is recommended. We should have two optimizers for D and G. Be careful to only take the gradient with respect to the variables to optimize. Namely
- $V(D)$: weights and biases of D
- $V(G)$: weights and biases of G

We provided the code for extracting these variables from the computation graph.

In [10]:
tvars = tf.trainable_variables()
disc_vars = [var for var in tvars if 'd_' in var.name]
gen_vars = [var for var in tvars if 'g_' in var.name]

In [11]:
optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [12]:
train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)

### 6. Train the model.
For each iteration, take some batch of MNIST. Generate a prior noise $z$ by `np.random.uniform(-1., 1., size=[batch_size, noise_dim])`. Feed the batch data and prior noise to the model to update the objective.

After some epochs of training, for each noise generated, get the output $x$ by the generator and plot it using matplotlib. This time we prepared the code for you but read through it to understand it. Then, change the variable names if they are different from yours.

In [13]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(tf.global_variables_initializer())
    
    for step in range(1, num_steps+1):

        batch_x, _ = mnist.train.next_batch(batch_size)
        # Generate noise to feed to the generator
        z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])
        # Train
        feed_dict = {disc_input: batch_x, gen_input: z}
        _, _, gl, dl = sess.run([train_gen, train_disc, gen_loss, disc_loss],
                                feed_dict=feed_dict)
        
        if step % 1000 == 0 or step == 1:
            print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (step, gl, dl))
    
        # Generate images from noise, using the generator network.
        if step % 10000 == 0 or step == 1:
            f, a = plt.subplots(4, 10, figsize=(10, 4))
            for i in range(10):
                # Noise input.
                z = np.random.uniform(-1., 1., size=[4, noise_dim])
                g = sess.run([gen_sample], feed_dict={gen_input: z})
                g = np.reshape(g, newshape=(4, 28, 28, 1))
                # Reverse colours for better display
                g = -1 * (g - 1)
                for j in range(4):
                    # Generate image from noise. Extend to 3 channels for matplot figure.
                    img = np.reshape(np.repeat(g[j][:, :, np.newaxis], 3, axis=2),
                                     newshape=(28, 28, 3))
                    a[j][i].imshow(img)

            plt.draw()
            print('gan'+str(step)+'.png')
            plt.savefig('gan'+str(step)+'.png')

Step 1: Generator Loss: 1.104980, Discriminator Loss: 1.427862
gan1.png


KeyboardInterrupt: 

### 7. [Optional] use TensorBoard to check the computation graph and loss.
You might want to read about [variable sharing](https://www.tensorflow.org/versions/r1.1/programmers_guide/variable_scope) and [variable scope](https://stackoverflow.com/questions/35919020/whats-the-difference-of-name-scope-and-a-variable-scope-in-tensorflow).

This might be a bit more involved than the previous steps...

Ok, this was probably the hardest section so far since there's less hand holding. But if you could complete this exercise, this means that you can build reasonably sophisticated neural network models in TensorFlow! Look back and see how far you got :)

Check [this](https://github.com/tensorflow/models/blob/master/research/gan/tutorial.ipynb) out if you're more interested in GANs.

Thanks for completing this workshop. If you liked it, please `star` this repo, so that more and more people can learn about TensorFlow! Feedback is always welcome!
